# Build Deep Research

This notebook walks through a naive implementation of DeepResearch like behaviour using ADK.


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Build_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FBuild_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Build_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/blob/main/Build_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Pili Hu](https://github.com/hupili)|
| Reviewers(s) | Nil |
| Last updated | 2025-06-08 |
| google-adk | 1.1.1 |
| vertexai | 1.93.1 |

## Environment

In [1]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [2]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [7]:
!pip install deprecated
!pip install google-adk==1.1.1

In [11]:
from google import adk

In [12]:
adk.__version__

'1.1.1'

## Util functions

In [13]:
from vertexai import agent_engines
from vertexai.preview import reasoning_engines

In [14]:
def delete(resource_id: str) -> None:
    remote_agent = agent_engines.get(resource_id)
    remote_agent.delete(force=True)
    print(f"Deleted remote agent: {resource_id}")

def list_agents() -> None:
    remote_agents = agent_engines.list()
    TEMPLATE = '''
{agent.name} ("{agent.display_name}")
- Create time: {agent.create_time}
- Update time: {agent.update_time}
'''
    remote_agents_string = '\n'.join(TEMPLATE.format(agent=agent) for agent in remote_agents)
    print(f"All remote agents:\n{remote_agents_string}")

In [15]:
list_agents()

All remote agents:

8348169577238102016 ("")
- Create time: 2025-06-03 06:09:08.152391+00:00
- Update time: 2025-06-03 06:13:15.300308+00:00


5147799092037943296 ("")
- Create time: 2025-06-02 14:14:28.600149+00:00
- Update time: 2025-06-02 14:18:55.955685+00:00


2164164338904989696 ("")
- Create time: 2025-06-02 11:57:47.050939+00:00
- Update time: 2025-06-02 12:01:18.059991+00:00


6087608854902734848 ("DICE_AGENT")
- Create time: 2025-04-25 08:19:20.817490+00:00
- Update time: 2025-04-25 08:22:05.936293+00:00


7916070303615156224 ("llm_auditor")
- Create time: 2025-04-25 07:13:25.339138+00:00
- Update time: 2025-04-25 07:18:12.898631+00:00


5307078744484085760 ("llm_auditor")
- Create time: 2025-04-09 08:22:36.179879+00:00
- Update time: 2025-04-09 08:25:42.734584+00:00


8402318325882814464 ("")
- Create time: 2025-03-28 08:42:23.121480+00:00
- Update time: 2025-03-28 08:46:29.629436+00:00


66410502317670400 ("my-awesome-agent")
- Create time: 2025-03-07 01:22:05.930759+00:00


In [16]:
def call_remote_agent_engine_app(app, user_id, session_id, query):
  response_text = ''
  for event in app.stream_query(
      user_id=user_id,
      session_id=session_id,
      message=query,
  ):
      print(event)
      all_parts = event.get('content', {}).get('parts', [])
      for part in all_parts:
          response_text += (part.get('text', ''))
  return response_text

## Simple Search Agent

In [17]:
from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

MODEL = "gemini-2.0-flash"
APP_NAME = "simple_search_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "simple_search_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

app = reasoning_engines.AdkApp(
    agent=simple_search_agent,
    enable_tracing=True,
)

In [18]:
session = app.create_session(user_id=USER_ID)
session

Session(id='d762d110-5a21-48e2-b393-0818e3f2913a', app_name='default-app-name', user_id='user12345', state={}, events=[], last_update_time=1749356030.1446936)

In [19]:
call_remote_agent_engine_app(app, USER_ID, session.id, "what is the weather in New York")

{'content': {'parts': [{'text': 'The weather in New York, NY is currently cloudy with a temperature of 68°F (20°C), but it feels like 76°F (24°C) due to 84% humidity. There is a 0% chance of rain.\n\nThe forecast for the next 10 days is:\n\n*   **Sunday:** Scattered showers during the day and light rain at night (20% chance of rain during the day and 45% at night). Temperatures between 60°F (16°C) and 73°F (23°C).\n*   **Monday:** Light rain during the day and cloudy at night (20% chance of rain). Temperatures between 62°F (17°C) and 68°F (20°C).\n*   **Tuesday:** Light rain during the day and clear at night (25% chance of rain during the day and 10% at night). Temperatures between 64°F (18°C) and 76°F (24°C).\n*   **Wednesday:** Sunny during the day and clear at night (5% chance of rain during the day). Temperatures between 68°F (20°C) and 83°F (28°C).\n*   **Thursday:** Partly cloudy (20% chance of rain during the day and 10% at night). Temperatures between 69°F (21°C) and 87°F (31°C

'The weather in New York, NY is currently cloudy with a temperature of 68°F (20°C), but it feels like 76°F (24°C) due to 84% humidity. There is a 0% chance of rain.\n\nThe forecast for the next 10 days is:\n\n*   **Sunday:** Scattered showers during the day and light rain at night (20% chance of rain during the day and 45% at night). Temperatures between 60°F (16°C) and 73°F (23°C).\n*   **Monday:** Light rain during the day and cloudy at night (20% chance of rain). Temperatures between 62°F (17°C) and 68°F (20°C).\n*   **Tuesday:** Light rain during the day and clear at night (25% chance of rain during the day and 10% at night). Temperatures between 64°F (18°C) and 76°F (24°C).\n*   **Wednesday:** Sunny during the day and clear at night (5% chance of rain during the day). Temperatures between 68°F (20°C) and 83°F (28°C).\n*   **Thursday:** Partly cloudy (20% chance of rain during the day and 10% at night). Temperatures between 69°F (21°C) and 87°F (31°C).\n*   **Friday:** Mostly cloud

In [20]:
call_remote_agent_engine_app(app, USER_ID, session.id, "is earth square?")

{'content': {'parts': [{'text': "No, the Earth is not square. The ancient Greeks were the first to document the concept that the Earth is spherical around the 5th century BC. In the 3rd century BC, it was established as a physical fact.\n\nHere's why we know the Earth is not square:\n\n*   **Ships disappearing hull first:** When a ship sails away from you, it doesn't just get smaller and smaller until it disappears. Instead, the hull of the ship disappears first, followed by the mast. This is because the Earth is curved, and the hull goes below the horizon before the mast does.\n*   **Lunar eclipses:** During a lunar eclipse, the Earth passes between the Sun and the Moon, casting a shadow on the Moon. This shadow is always round, which wouldn't be the case if the Earth were flat or square.\n*   **Different constellations:** Different constellations are visible from different parts of the Earth. For example, Polaris (the North Star) is higher in the sky in the Northern Hemisphere than i

"No, the Earth is not square. The ancient Greeks were the first to document the concept that the Earth is spherical around the 5th century BC. In the 3rd century BC, it was established as a physical fact.\n\nHere's why we know the Earth is not square:\n\n*   **Ships disappearing hull first:** When a ship sails away from you, it doesn't just get smaller and smaller until it disappears. Instead, the hull of the ship disappears first, followed by the mast. This is because the Earth is curved, and the hull goes below the horizon before the mast does.\n*   **Lunar eclipses:** During a lunar eclipse, the Earth passes between the Sun and the Moon, casting a shadow on the Moon. This shadow is always round, which wouldn't be the case if the Earth were flat or square.\n*   **Different constellations:** Different constellations are visible from different parts of the Earth. For example, Polaris (the North Star) is higher in the sky in the Northern Hemisphere than it is in the Southern Hemisphere.

## Search multiple questions


In [21]:
from google.adk.tools.agent_tool import AgentTool

# Instantiate again because each agent tree shall have unique objects.
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

multiple_search_agent = Agent(
    model=MODEL,
    name="research_agent",
    description="Get answers to multiple questions one by one.",
    instruction="Observe the pending questions in the chat log and call the simple_search_agent to conduct search and give answers to each question. Present the search result in a markdown, one section per question and answer",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[AgentTool(simple_search_agent)],
)

app = reasoning_engines.AdkApp(
    agent=multiple_search_agent,
    enable_tracing=True,
)

session = app.create_session(user_id=USER_ID)

call_remote_agent_engine_app(app, USER_ID, session.id, """
- Is Earth square?
- Is Moon flat?
- Is Sun cold?
""")

{'content': {'parts': [{'function_call': {'id': 'adk-c533a0b5-56ab-4316-877b-d57402b957ee', 'args': {'request': 'Is Earth square?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-22e4dc86-b35e-42b2-84c8-c64ba60301ec', 'args': {'request': 'Is Moon flat?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-ee995d30-b5cd-4585-aa0c-7eda31b33262', 'args': {'request': 'Is Sun cold?'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 30, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 30}], 'prompt_token_count': 107, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 107}], 'total_token_count': 137, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-6aa50a75-50e0-4b69-addd-cea17a8081ba', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids'

"Here are the answers to your questions:\n\n**Is Earth square?**\n\nNo, the Earth is not square. It is more accurately described as an oblate spheroid or an ellipsoid, which means it's like a sphere that is slightly flattened at the poles and bulging at the equator due to its rotation.\n\n**Is Moon flat?**\n\nNo, the moon is not flat. It is a sphere, though not a perfect one, and is often described as egg-shaped or an oblate spheroid.\n\n**Is Sun cold?**\n\nNo, the Sun is not cold. The core is the hottest part, reaching around 27 million degrees Fahrenheit (15 million degrees Celsius), while the surface is around 10,000 degrees Fahrenheit (5,500 degrees Celsius).\n"

In [ ]:
# One sample data related to function call
# data = {
#     'content': {
#         'parts': [
#             {'function_call':
#              {'id': 'adk-86084634-fdb5-45ff-be8f-ef3fc5866a35',
#               'args': {'request': 'Is Earth square?'},
#               'name': 'simple_search_agent'}},
#             {'function_call':
#              {'id': 'adk-482d0901-8ff9-4f48-bf2c-03539b6dc35a',
#               'args': {'request': 'Is Moon flat?'}, 'name': 'simple_search_agent'}},
#             {'function_call':
#              {'id': 'adk-a8b821f0-b93c-470e-bf41-fa89e7be6d9c',
#               'args': {'request': 'Is Sun cold?'}, 'name': 'simple_search_agent'}}

## Generate the research plan

In [30]:
DEEP_RESEARCH_PLAN_INSTRUCTION = """
Instruction for a Deep Research Plan Agent

As a Deep Research Plan Agent, your primary objective is to generate a comprehensive and insightful research plan. This plan will be presented as a markdown bullet point list of questions. The goal is to not only address the user's explicit query but also to anticipate and explore related avenues of inquiry that would be of interest for a thorough understanding of the topic. Your output should guide the user through a structured and deep exploration of their subject.

Here is the process to follow:

Deconstruct the Core Topic:

What is the fundamental subject of the user's request?
What are the primary keywords and concepts?
What is the implicit goal of the user's research? (e.g., to understand a concept, to make a decision, to compare and contrast, to solve a problem).
Generate Foundational Questions:

Start with the basics to establish a strong foundation.
What is the definition of the core topic?
What is its history and origin?
What are the key principles and theories that underpin it?
Who are the seminal thinkers, researchers, or figures associated with this topic?
Explore Deeper, More Nuanced Aspects:

Move beyond the surface-level inquiries.
What are the current debates or controversies surrounding this topic?
What are the different schools of thought or perspectives?
What are the key components, mechanisms, or processes involved? How do they interact?
What are the less obvious or often overlooked aspects of this topic?
Investigate Connections and Relationships:

Think about how the topic connects to broader fields and concepts.
How does this topic relate to other relevant disciplines or areas of study?
What are the upstream causes and downstream effects of the subject?
How has this topic influenced, or been influenced by, historical, social, technological, or economic factors?
Consider Practical Applications and Implications:

Explore the real-world significance of the topic.
What are the practical applications or use cases?
What are the ethical, social, or economic implications?
Who are the key stakeholders affected by or involved in this topic?
Look to the Future:

Encourage forward-thinking inquiry.
What are the emerging trends and future directions of research in this area?
What are the unanswered questions or existing gaps in knowledge?
What are the potential future impacts or developments related to this topic?
Synthesize and Structure the Plan:

Organize the generated questions into a logical flow. A good structure might be:
Foundational Understanding
In-Depth Analysis
Context and Connections
Practical Implications
Future Outlook
Present the final output as a clean, easy-to-read markdown bullet point list.
Use sub-bullets to group related questions and create a hierarchical structure.
Ensure the questions are open-ended and encourage critical thinking rather than simple fact-finding.
By following these instructions, you will create a research plan that is not only deep and thoughtful but also anticipates the user's latent interests, guiding them toward a truly comprehensive understanding of their chosen topic.

IMPORTANT:
- Output only 10 key questions related to this topic in concise English.
- Output as a single layer markdown list.
"""

In [31]:
# Instantiate again because each agent tree shall have unique objects.
plan_agent = Agent(
    model=MODEL,
    name="plan_agent",
    description="Agent to generate a research plan based on the question or topic given by the user",
    instruction=DEEP_RESEARCH_PLAN_INSTRUCTION,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

In [32]:
app = reasoning_engines.AdkApp(
    agent=plan_agent,
    enable_tracing=True,
)

session = app.create_session(user_id=USER_ID)

response_text = call_remote_agent_engine_app(app, USER_ID, session.id, """AI Agent's impact on labour market""")

{'content': {'parts': [{'text': '*   What is the current state of AI agent technology and its deployment across various industries?\n*   How are AI agents currently being used to automate tasks previously performed by human workers?\n*   What specific job roles and industries are most vulnerable to displacement by AI agents?\n*   What new job roles and opportunities are emerging as a result of the development and deployment of AI agents?\n*   How can education and training programs be adapted to prepare workers for the changing demands of the labor market in the age of AI agents?\n*   What are the potential economic consequences of widespread AI agent adoption, including effects on wages, income inequality, and overall productivity?\n*   How can governments and policymakers mitigate the negative impacts of AI agent-driven job displacement and ensure a just transition for affected workers?\n*   What are the ethical considerations surrounding the use of AI agents in the workplace, includ

In [33]:
# prompt: display response_text as markdown

from IPython.display import display, Markdown

display(Markdown(response_text))

*   What is the current state of AI agent technology and its deployment across various industries?
*   How are AI agents currently being used to automate tasks previously performed by human workers?
*   What specific job roles and industries are most vulnerable to displacement by AI agents?
*   What new job roles and opportunities are emerging as a result of the development and deployment of AI agents?
*   How can education and training programs be adapted to prepare workers for the changing demands of the labor market in the age of AI agents?
*   What are the potential economic consequences of widespread AI agent adoption, including effects on wages, income inequality, and overall productivity?
*   How can governments and policymakers mitigate the negative impacts of AI agent-driven job displacement and ensure a just transition for affected workers?
*   What are the ethical considerations surrounding the use of AI agents in the workplace, including issues of bias, transparency, and accountability?
*   How do different countries and regions vary in their approaches to regulating and adapting to the impact of AI agents on the labor market?
*   What are the long-term societal implications of a labor market increasingly shaped by AI agents, including changes in work culture, social structures, and human well-being?


## Puting it all together

In [34]:
# Instantiate again because each agent tree shall have unique objects.
plan_agent = Agent(
    model=MODEL,
    name="plan_agent",
    description="Agent to generate a research plan based on the question or topic given by the user",
    instruction=DEEP_RESEARCH_PLAN_INSTRUCTION,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

multiple_search_agent = Agent(
    model=MODEL,
    name="research_agent",
    description="Get answers to multiple questions one by one.",
    instruction="Observe the pending questions in the chat log and call the simple_search_agent to conduct search and give answers to each question. Present the search result in a markdown, one section per question and answer",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[AgentTool(simple_search_agent)],
)

report_agent = Agent(
    model=MODEL,
    name="report_agent",
    description="Agent to generate a markdown research report from the questions and their search results from the Internet",
    instruction="Summarize the research results and present research report in markdown format",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

from google.adk.agents.sequential_agent import SequentialAgent

root_agent = SequentialAgent(
    name="deep_researcher",
    description="This is a deep research that makes research plan, use Google search to gather information and generate a report.",
    sub_agents=[plan_agent, multiple_search_agent, report_agent],
)

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [35]:
session = app.create_session(user_id=USER_ID)

response_text = call_remote_agent_engine_app(app, USER_ID, session.id, """AI Agent's impact on labour market""")

{'content': {'parts': [{'text': '- What specific tasks and industries are most likely to be automated or augmented by AI agents?\n- How might AI agents reshape job roles and skill requirements across different sectors?\n- What strategies can be implemented to mitigate potential job displacement caused by AI agents?\n- How can education and training programs adapt to prepare workers for the changing demands of the labor market?\n- What new job opportunities might emerge as a direct or indirect result of the widespread adoption of AI agents?\n- How do the economic benefits of AI agent adoption compare to the potential social costs related to employment?\n- What are the ethical considerations surrounding the use of AI agents in the workplace, particularly regarding bias and fairness?\n- How might government policies and regulations need to evolve to address the labor market implications of AI agents?\n- What are the potential long-term effects of AI agents on wage levels and income inequa

{'content': {'parts': [{'text': "Okay, I will use the `simple_search_agent` to conduct search and give answers to each question about AI Agent's impact on labour market.\n\n"}, {'function_call': {'id': 'adk-8c76eb23-60ae-4714-9c02-d65b92fae7fc', 'args': {'request': 'What specific tasks and industries are most likely to be automated or augmented by AI agents?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-46c913bb-bf55-4678-b3a2-353774dfef0c', 'args': {'request': 'How might AI agents reshape job roles and skill requirements across different sectors?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-d0bff079-dca9-47fd-95a6-dd29e751d7c0', 'args': {'request': 'What strategies can be implemented to mitigate potential job displacement caused by AI agents?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-1fff1927-0f0e-4527-9ebe-513f06b4b0a3', 'args': {'request': 'How can education and training programs adapt to prepare workers for the changi

In [36]:
from IPython.display import display, Markdown

display(Markdown(response_text))

- What specific tasks and industries are most likely to be automated or augmented by AI agents?
- How might AI agents reshape job roles and skill requirements across different sectors?
- What strategies can be implemented to mitigate potential job displacement caused by AI agents?
- How can education and training programs adapt to prepare workers for the changing demands of the labor market?
- What new job opportunities might emerge as a direct or indirect result of the widespread adoption of AI agents?
- How do the economic benefits of AI agent adoption compare to the potential social costs related to employment?
- What are the ethical considerations surrounding the use of AI agents in the workplace, particularly regarding bias and fairness?
- How might government policies and regulations need to evolve to address the labor market implications of AI agents?
- What are the potential long-term effects of AI agents on wage levels and income inequality?
- How does the impact of AI agents on the labor market differ across various countries or regions, and what factors contribute to these differences?
Okay, I will use the `simple_search_agent` to conduct search and give answers to each question about AI Agent's impact on labour market.

Okay, I have the search results for each of your questions. Here's a summary presented in markdown format:

### What specific tasks and industries are most likely to be automated or augmented by AI agents?

AI agents are likely to automate tasks such as data entry, repetitive administrative work, customer interactions, lead generation, and HR tasks. Industries with high automation potential include customer service, finance, healthcare, logistics, manufacturing, marketing, retail, high tech, and education. Specific roles at risk include data entry clerks, customer service representatives, cashiers, telemarketers, and assembly line operators.

### How might AI agents reshape job roles and skill requirements across different sectors?

AI agents will automate routine tasks, leading to job displacement and the creation of new roles in AI management, machine learning, data analysis, and AI ethics. Existing jobs will be transformed to incorporate AI tools, enhancing productivity. Skill requirements will evolve, with increased demand for technical skills like data analytics and soft skills like critical thinking and communication. Industries like manufacturing, healthcare, and customer service will see significant changes in job roles.

### What strategies can be implemented to mitigate potential job displacement caused by AI agents?

Strategies to mitigate job displacement include reskilling and upskilling programs, promoting AI-human collaboration through job redesign, encouraging innovation and entrepreneurship in AI-related fields, implementing supportive policies like social safety nets and wage subsidies, and adopting ethical AI deployment guidelines. Transparent communication about AI's impact is also crucial.

### How can education and training programs adapt to prepare workers for the changing demands of the labor market?

Education and training programs should focus on skills-based learning, regularly update curricula to align with job market demands, embrace lifelong learning, integrate technology like AI tools, strengthen industry partnerships, and support career transitions. They should also recognize the importance of skills and credentials over traditional degrees.

### What new job opportunities might emerge as a direct or indirect result of the widespread adoption of AI agents?

New job opportunities will emerge directly in areas like AI agent development, machine learning engineering, AI training, AI ethics, and AI integration consulting. Indirectly, there will be increased demand for data analysts and creative roles in marketing, as well as growth in education and power systems engineering.

### How do the economic benefits of AI agent adoption compare to the potential social costs related to employment?

The economic benefits of AI adoption include increased productivity, cost reduction, and revenue growth. However, the social costs include job displacement, the need for reskilling, potential wage stagnation, and increased income inequality. Balancing these requires proactive measures like investments in reskilling, new social safety nets, and thoughtful policy and regulation.

### What are the ethical considerations surrounding the use of AI agents in the workplace, particularly regarding bias and fairness?

Ethical considerations include bias and fairness in AI decision-making, the need for transparency and explainability in AI processes, accountability for AI actions, job displacement, and data privacy. Addressing these requires rebalancing training data, auditing AI systems for bias, and establishing ethical frameworks and legal compliance.

### How might government policies and regulations need to evolve to address the labor market implications of AI agents?

Government policies and regulations need to evolve to protect workers' rights, address job displacement through training programs, promote fairness in hiring and promotion, ensure algorithmic transparency, and support collective bargaining. Establishing AI standards boards and defining clear accountability for AI systems are also important.

### What are the potential long-term effects of AI agents on wage levels and income inequality?

The long-term effects are complex, with potential negative impacts including job displacement, wage stagnation, and increased income inequality. However, there are also potential positive impacts such as increased productivity, new job creation, and improved job quality. The actual outcome will depend on policy choices, the speed of AI adoption, and the adaptability of workers.

### How does the impact of AI agents on the labor market differ across various countries or regions, and what factors contribute to these differences?

The impact of AI varies across countries due to differing levels of AI exposure and complementarity, economic structure, workforce skills, government policies, technological infrastructure, and cultural factors. Advanced economies with highly skilled workforces and strong AI strategies are better positioned to benefit, while emerging economies may face greater challenges related to job displacement and skills gaps.
## AI Agent's Impact on the Labour Market: A Research Report

### Introduction

This report summarizes the potential impacts of Artificial Intelligence (AI) agents on the labor market, based on a review of recent research. It addresses key questions regarding automation, job roles, mitigation strategies, education, new opportunities, economic considerations, ethics, policy, wages, inequality, and global variations.

### 1. Automation and Affected Industries

**What specific tasks and industries are most likely to be automated or augmented by AI agents?**

AI agents are likely to automate tasks such as:

*   Data Entry and Processing
*   Repetitive Administrative Tasks
*   Customer Interactions
*   Lead Generation and Qualification
*   Sales and Marketing Automation
*   HR Tasks
*   Data Analysis

Industries with high automation potential include:

*   Customer Service
*   Finance
*   Healthcare
*   Logistics and Warehousing
*   Manufacturing
*   Marketing
*   Retail and Consumer Packaged Goods (CPG)
*   High Tech
*   Education

Specific roles at risk include:

*   Data Entry Clerks
*   Customer Service Representatives
*   Cashiers
*   Telemarketers
*   Receptionists
*   Assembly Line Operators
*   Quality Inspectors
*   Bank Tellers
*   Stock Traders
*   Loan Officers

### 2. Reshaping Job Roles and Skill Requirements

**How might AI agents reshape job roles and skill requirements across different sectors?**

AI agents will automate routine tasks, leading to both job displacement and the creation of new roles. Existing jobs will be transformed to incorporate AI tools, enhancing productivity. Skill requirements will evolve, with increased demand for technical skills (data analytics, machine learning) and soft skills (critical thinking, communication). Industries like manufacturing, healthcare, and customer service will see significant changes.

### 3. Mitigating Job Displacement

**What strategies can be implemented to mitigate potential job displacement caused by AI agents?**

Strategies include:

*   Reskilling and upskilling programs
*   Promoting AI-human collaboration through job redesign
*   Encouraging innovation and entrepreneurship in AI-related fields
*   Implementing supportive policies like social safety nets and wage subsidies
*   Adopting ethical AI deployment guidelines
*   Transparent communication about AI's impact

### 4. Adapting Education and Training

**How can education and training programs adapt to prepare workers for the changing demands of the labor market?**

Education and training programs should:

*   Focus on skills-based learning
*   Regularly update curricula to align with job market demands
*   Embrace lifelong learning
*   Integrate technology like AI tools
*   Strengthen industry partnerships
*   Support career transitions
*   Recognize the importance of skills and credentials over traditional degrees

### 5. New Job Opportunities

**What new job opportunities might emerge as a direct or indirect result of the widespread adoption of AI agents?**

*Directly Related to AI Agents:*

*   AI Agent Developers
*   Machine Learning Engineers
*   AI Trainers/Teachers
*   AI Ethicists/Policy Specialists
*   AI Integration Consultants
*   AI System Auditors
*   AI Maintenance Professionals
*   Prompt Engineers
*   AI Personality Designers

*Indirectly Related to AI Agents:*

*   Data Analysts
*   Creative roles in marketing and branding
*   University and Higher Education Teachers
*   Power Systems Engineers

### 6. Economic Benefits vs. Social Costs

**How do the economic benefits of AI agent adoption compare to the potential social costs related to employment?**

The economic benefits of AI adoption include increased productivity, cost reduction, and revenue growth. However, the social costs include job displacement, the need for reskilling, potential wage stagnation, and increased income inequality. Balancing these requires proactive measures such as investments in reskilling, new social safety nets, and thoughtful policy and regulation.

### 7. Ethical Considerations

**What are the ethical considerations surrounding the use of AI agents in the workplace, particularly regarding bias and fairness?**

Ethical considerations include:

*   Bias and fairness in AI decision-making
*   The need for transparency and explainability in AI processes
*   Accountability for AI actions
*   Job displacement
*   Data privacy

Addressing these requires rebalancing training data, auditing AI systems for bias, and establishing ethical frameworks and legal compliance.

### 8. Government Policies and Regulations

**How might government policies and regulations need to evolve to address the labor market implications of AI agents?**

Government policies and regulations need to:

*   Protect workers' rights
*   Address job displacement through training programs
*   Promote fairness in hiring and promotion
*   Ensure algorithmic transparency
*   Support collective bargaining
*   Establish AI standards boards
*   Define clear accountability for AI systems

### 9. Long-Term Effects on Wages and Income Inequality

**What are the potential long-term effects of AI agents on wage levels and income inequality?**

The long-term effects are complex, with potential negative impacts including job displacement, wage stagnation, and increased income inequality. However, there are also potential positive impacts such as increased productivity, new job creation, and improved job quality. The actual outcome will depend on policy choices, the speed of AI adoption, and the adaptability of workers.

### 10. Global Variations

**How does the impact of AI agents on the labor market differ across various countries or regions, and what factors contribute to these differences?**

The impact of AI varies across countries due to:

*   Differing levels of AI exposure and complementarity
*   Economic structure
*   Workforce skills
*   Government policies
*   Technological infrastructure
*   Cultural factors

Advanced economies with highly skilled workforces and strong AI strategies are better positioned to benefit, while emerging economies may face greater challenges related to job displacement and skills gaps.

### Conclusion

AI agents present both opportunities and challenges for the labor market. Proactive and thoughtful policies, combined with a focus on education, ethics, and worker support, will be crucial to maximizing the benefits of AI while mitigating its potential negative consequences.


## Bonus: Finish the remaining researches

Note that the above result has many open questions not yet researched. Try to build an agent tree that completes the research. Some possible directions:

- Use LoopAgent, refer to the QuickStart notebook of how to exit_loop.
- Optimize the prompt of multiple_search agent so it conducts more thorough searches.

## Bonus: Format with grounding data

The simple_search_agent returns not only the search results summary, but also the grounding web pages and references / citations.